### KeyBERTを用いた類似説明性の付与
https://maartengr.github.io/KeyBERT/guides/quickstart.html

In [1]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

# MODEL_NAME = "all-MiniLM-L6-v2"
# MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
# MODEL_NAME = "sonoisa/sentence-luke-japanese-base-lite"
MODEL_NAME = "sonoisa/sentence-bert-base-ja-mean-tokens-v2"
sentence_model = SentenceTransformer(MODEL_NAME)

In [2]:
# https://jgoodtech.smrj.go.jp/pub/ja/journal/benchmark/
# 大和化学工業株式会社
needs_text = """
超高性能特殊樹脂の加工ができる企業を探しており、ジェグテックのニーズ機能にて対応できる企業を探していた。
      """
seeds_text = """
プラスチック製品の企画・製造・販売を行う。
顧客の様々な要望に対し、柔軟に応えられる高い技術力を持っており、本ニーズへの提案を行った。
      """

# ハウス食品グループ本社株式会社 山伝製紙株式会社
needs_text = """
SDGsの取り組みとして、新たな価値を生み出す食品残渣を活用したアップサイクルに向けたパートナー企業を探していた。
ウコンエキスドリンク「ウコンの力」の製造過程で出るウコンの搾りかすを活用した「クルクミンの色を生かした紙の作製」のアイデアに協力してくれる企業を探していた。
      """
seeds_text = """
伝統的な越前和紙の製造技術
機械抄きによってさまざまな機能紙などの製造技術
紙を抄いた上での日焼け防止の薬品コーティング技術
      """

# ダイカテック株式会社
needs_text = """
自社の持つ粉体付着防止技術（F研磨）により、「泡立ちのよいビアカップ」を開発した。
大手飲料メーカーなどにコンタクトを取ったものの、取引に至らず試行錯誤していた。
斬新なアイディアを求めて、ジェグテックのニーズ機能にて、商品企画パートナーを募集した。
      """
seeds_text = """
自社でECサイトを運営し、デザイン企画・製作を手掛ける中小企業。
アイディアのある企画をベースに、地域や企業ブランディングを得意としており、本ニーズに提案を行った。
      """

# 株式会社イマイ
needs_text = """
東南アジアへの取引拡大を目指し、東南アジアのローカル企業と出会うため、ジェグテックを活用した。
中小機構が主催した商談会に参加し、ベトナム企業との商談を行った。
      """
seeds_text = """
ベトナム企業（G社）：現地の食品メーカー
ベトナム企業（H社）：商社機能を合わせ持つ、現地の食品メーカー
      """

# ダイカテック株式会社
needs_text = """
金属異物の検出・除去装置を求めており、対応可能性のありそうなジェグテック登録企業14社に対してニーズを発信した。
      """
seeds_text = """
粉体特性を熟知した金属異物除去装置メーカー。
他社との違いや、自社が実現可能なこと、自社の強みを活かした提案を実施した。
      """

raw_text = f"{needs_text}\n{seeds_text}"
print(raw_text)

TOP_K = 20


金属異物の検出・除去装置を求めており、対応可能性のありそうなジェグテック登録企業14社に対してニーズを発信した。
      

粉体特性を熟知した金属異物除去装置メーカー。
他社との違いや、自社が実現可能なこと、自社の強みを活かした提案を実施した。
      


#### テキストを連結してキーワードを探す

In [3]:
kw_model = KeyBERT(model=sentence_model)

# こっちで取っても可
# from transformers import BertJapaneseTokenizer 
# tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
tokenized_text = sentence_model.tokenizer.tokenize(raw_text)
input_text = ' '.join(tokenized_text)
print(input_text)

keywords = kw_model.extract_keywords(
    input_text, 
    top_n = TOP_K, 
    keyphrase_ngram_range=(1, 2)
    )
keywords

金属 異 ##物 の 検出 ・ 除去 装置 を 求め て おり 、 対応 可能 性 の あり そう な ジェ ##グ ##テック 登録 企業 14 社 に対して ニーズ を 発信 し た 。 粉 体 特性 を 熟 ##知 し た 金属 異 ##物 除去 装置 メーカー 。 他社 と の 違い や 、 自社 が 実現 可能 な こと 、 自社 の 強 ##み を 活かし た 提案 を 実施 し た 。


[('企業 14', 0.5452),
 ('メーカー 他社', 0.5342),
 ('メーカー', 0.5188),
 ('特性 金属', 0.5093),
 ('こと 自社', 0.5067),
 ('他社 違い', 0.4916),
 ('企業', 0.4832),
 ('自社 活かし', 0.4796),
 ('違い 自社', 0.4768),
 ('装置 メーカー', 0.4688),
 ('他社', 0.4644),
 ('金属 検出', 0.4611),
 ('自社', 0.4572),
 ('金属 除去', 0.437),
 ('金属', 0.4345),
 ('ジェ テック', 0.4227),
 ('登録 企業', 0.4214),
 ('テック', 0.4142),
 ('テック 登録', 0.4002),
 ('特性', 0.396)]

#### ニーズとシーズを分けてキーワードを探す

In [4]:
tokenized_text = sentence_model.tokenizer.tokenize(needs_text)
input_text = ' '.join(tokenized_text)

keywords = kw_model.extract_keywords(
    input_text, 
    top_n = TOP_K, 
    keyphrase_ngram_range=(1, 2), 
    seed_keywords=sentence_model.tokenizer.tokenize(seeds_text), 
#     candidates=tokenized_text, # candidatesの挙動がよくわからない
    )
keywords

[('企業 14', 0.6349),
 ('テック 登録', 0.529),
 ('ジェ テック', 0.5218),
 ('装置 求め', 0.5047),
 ('テック', 0.4887),
 ('14 に対して', 0.487),
 ('14', 0.4771),
 ('企業', 0.4631),
 ('金属', 0.4577),
 ('金属 検出', 0.4563),
 ('ニーズ 発信', 0.4509),
 ('登録 企業', 0.4452),
 ('に対して ニーズ', 0.4353),
 ('対応 可能', 0.4297),
 ('対応', 0.3834),
 ('検出', 0.381),
 ('可能 あり', 0.3703),
 ('除去 装置', 0.37),
 ('検出 除去', 0.3674),
 ('求め おり', 0.3654)]

#### ニーズとシーズを分けてキーワードを探す（n-gramがめんどい）
1. needs_textの文章ベクトルを作成
2. needs_textをTokenizeし、単語ベクトルを作成
3. seeds_textの文章ベクトルを作成
4. seeds_textをTokenizeし、単語ベクトルを作成
5. 1.と4.のコサイン類似度を計算
6. 2.と3.のコサイン類似度を計算
7. 5.と6.の結果をよろしく結合する

In [5]:
from scipy import spatial
import torch
needs_output = sentence_model.encode(needs_text,output_value=None)
needs_embeddings = needs_output['token_embeddings']
seeds_embeddings = sentence_model.encode(seeds_text)

# needs_output = sentence_model.encode(seeds_text,output_value=None)
# needs_embeddings = needs_output['token_embeddings']
# seeds_embeddings = sentence_model.encode(needs_text)

# needs_output = sentence_model.encode(raw_text,output_value=None)
# needs_embeddings = needs_output['token_embeddings']
# seeds_embeddings = sentence_model.encode(raw_text)

distances = 1-spatial.distance.cdist(seeds_embeddings.reshape(1,-1), needs_embeddings, 'cosine')
results = torch.topk(torch.tensor(distances[0]), TOP_K)
print(results)

sentence_model.tokenizer.decode(needs_output['input_ids'][results.indices]).split()

torch.return_types.topk(
values=tensor([0.5266, 0.5135, 0.5117, 0.5037, 0.5035, 0.4863, 0.4831, 0.4821, 0.4817,
        0.4798, 0.4660, 0.4636, 0.4571, 0.4563, 0.4562, 0.4553, 0.4438, 0.4428,
        0.4248, 0.4226], dtype=torch.float64),
indices=tensor([ 4,  5,  3, 14, 17, 15,  8,  9, 16,  1,  6, 27,  2,  7, 25, 19,  0, 20,
        13, 18]))


['の',
 '検出物',
 '対応',
 'の',
 '可能',
 '装置',
 'を',
 '性',
 '金属',
 '・',
 '社',
 '異',
 '除去',
 '企業',
 'そう',
 '[CLS]',
 'な',
 '、',
 'あり']

#### pke

In [6]:
import pke

# キーワード抽出を行う関数
def extract_keywords(text, model_name='MultipartiteRank'):
    if model_name=='MultipartiteRank':
        extractor = pke.unsupervised.MultipartiteRank()
        
        extractor.load_document(input=text, language='ja', normalization=None)
        extractor.candidate_selection(pos={'NOUN', 'PROPN', 'ADJ', 'NUM'})
        extractor.candidate_weighting(threshold=0.74)
        keywords = extractor.get_n_best(n=TOP_K)

    if model_name=='TopicRank':
        extractor = pke.unsupervised.TopicRank()
        
        extractor.load_document(input=text, language='ja', normalization=None)
        extractor.candidate_selection(pos={'NOUN', 'PROPN', 'ADJ', 'NUM'})
        extractor.candidate_weighting(threshold=0.74)
        keywords = extractor.get_n_best(n=TOP_K)
        
    if model_name=='TfIdf':
        extractor = pke.unsupervised.TfIdf()
        
        extractor.load_document(input=text, language='ja', normalization=None)
        extractor.candidate_selection(n=3)
        extractor.candidate_weighting()
        keywords = extractor.get_n_best(n=TOP_K)
    return [kw[0] for kw in keywords]

extract_keywords(raw_text, model_name='MultipartiteRank')

['金属 異物', '除去 装置', '粉体 特性', 'ニーズ', '金属 異物 除去 装置 メーカー', '実現 可能']

In [7]:
extract_keywords(raw_text, model_name='TfIdf')

['金属 異物',
 '除去 装置',
 '対応 可能',
 'あり そう',
 'ジェグテック',
 'ジェグテック 登録',
 'ジェグテック 登録 企業',
 '登録 企業',
 '登録 企業 14',
 '企業 14',
 'ニーズ',
 '粉体 特性',
 '金属 異物 除去',
 '異物 除去',
 '異物 除去 装置',
 '除去 装置 メーカー',
 '装置 メーカー',
 'メーカー',
 '実現 可能',
 '活かし']

#### tf-idf

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [' '.join(sentence_model.tokenizer.tokenize(raw_text))]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
results = torch.topk(torch.tensor(X.toarray()), TOP_K)
dict(zip(
    vectorizer.get_feature_names_out()[results.indices][0], 
    results.values[0].numpy()
        ))

{'可能': 0.30499714066520933,
 '自社': 0.30499714066520933,
 '装置': 0.30499714066520933,
 '金属': 0.30499714066520933,
 '除去': 0.30499714066520933,
 '14': 0.15249857033260467,
 'あり': 0.15249857033260467,
 'おり': 0.15249857033260467,
 'こと': 0.15249857033260467,
 'そう': 0.15249857033260467,
 'に対して': 0.15249857033260467,
 'ジェ': 0.15249857033260467,
 'テック': 0.15249857033260467,
 'ニーズ': 0.15249857033260467,
 'メーカー': 0.15249857033260467,
 '他社': 0.15249857033260467,
 '企業': 0.15249857033260467,
 '実施': 0.15249857033260467,
 '実現': 0.15249857033260467,
 '対応': 0.15249857033260467}